# Segmentation

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/segmentation](https://github.com/huseinzol05/Malaya/tree/master/example/segmentation).
    
</div>

<div class="alert alert-info">

This module trained on both standard and local (included social media) language structures, so it is save to use for both.
    
</div>

In [1]:
%%time

import malaya

CPU times: user 5.61 s, sys: 1.08 s, total: 6.69 s
Wall time: 7.36 s


Common problem for social media texts, there are missing spaces in the text, so text segmentation can help you,

1. huseinsukamakan ayam,dia sgtrisaukan -> husein suka makan ayam, dia sgt risaukan.
2. drmahathir sangat menekankan budaya budakzamansekarang -> dr mahathir sangat menekankan budaya budak zaman sekarang.
3. ceritatunnajibrazak -> cerita tun najib razak.
4. TunM sukakan -> Tun M sukakan.

Segmentation only,

1. Solve spacing error.
3. Not correcting any grammar.

In [2]:
string1 = 'huseinsukamakan ayam,dia sgtrisaukan'
string2 = 'drmahathir sangat menekankan budaya budakzamansekarang'
string3 = 'ceritatunnajibrazak'
string4 = 'TunM sukakan'
string_hard = 'IPOH-AhliDewanUndangan Negeri(ADUN) HuluKinta, MuhamadArafat Varisai Mahamadmenafikanmesejtularmendakwa beliau akan melompatparti menyokong UMNO membentuk kerajaannegeridiPerak.BeliauyangjugaKetua Penerangan Parti Keadilan Rakyat(PKR)Perak dalam satumesejringkaskepadaSinar Harian menjelaskan perkara itutidakbenarsama sekali.'
string_socialmedia = 'aqxsukalah apeyg tejadidekat mamattu'

### Viterbi algorithm

Commonly people use Viterbi algorithm to solve this problem, we also added viterbi using ngram from bahasa papers and wikipedia.

```python
def viterbi(max_split_length: int = 20, **kwargs):
    """
    Load Segmenter class using viterbi algorithm.

    Parameters
    ----------
    max_split_length: int, (default=20)
        max length of words in a sentence to segment
    validate: bool, optional (default=True)
        if True, malaya will check model availability and download if not available.

    Returns
    -------
    result : malaya.segmentation.SEGMENTER class
    """
```

In [3]:
viterbi = malaya.segmentation.viterbi()

In [4]:
%%time

viterbi.segment([string1, string2, string3, string4])

CPU times: user 60.7 ms, sys: 1.09 ms, total: 61.8 ms
Wall time: 62.2 ms


['husein suka makan ayam,dia sgt risau kan',
 'dr mahathir sangat mene kan kan budaya budak zaman sekarang',
 'cerita tu n najib razak',
 'Tun M suka kan']

In [5]:
%%time

viterbi.segment([string_hard, string_socialmedia])

CPU times: user 7.46 ms, sys: 355 µs, total: 7.81 ms
Wall time: 7.61 ms


['IPOH - Ahli Dewan Undangan Negeri(ADUN) Hulu Kinta, Muhamad Arafat Varisai Mahamadmenafikanmesejtularmendakwa belia u akan me lompat part i me nyo ko ng UMNO mem bentuk kerajaannegeridi Perak. Beliauyangjuga Ketua Penerangan Parti Keadilan Rakyat(PKR) Perak dalam satumesejringkaskepada Sinar Harian men jel ask an perkara it u tidak benar sama sekali.',
 'aq x suka lah ape yg te jadi dekat mama ttu']

### List available Transformer model

In [6]:
malaya.segmentation.available_transformer()

,Size (MB),Quantized Size (MB),Sequence Accuracy
small,42.7,13.1,0.8217
base,234.0,63.8,0.8759


### Load Transformer model

In [7]:
model = malaya.segmentation.transformer(model = 'small')

In [5]:
quantized_model = malaya.segmentation.transformer(model = 'small', quantized = True)

#### Predict

In [7]:
%%time

model.segment([string1, string2, string3, string4], beam_search = False)

CPU times: user 3 s, sys: 680 ms, total: 3.68 s
Wall time: 973 ms


['husein suka makan ayam, dia sgt risaukan Aku menangis',
 'dr mahathir sangat menekankan budaya budak zaman sekarang mahathir sangat menekankan budaya budak zaman sekarang mahathir sangat menekankan budaya budak zaman sekarang',
 'cerita tun najib razak',
 'Tun M sukakan Tun M sukakan Tun M sukakan',
 'IPOH - Ahli Dewan Undangan Negeri (ADUN) Hulu Kinta, Muhamad Arafat Varisai Mahamad menafikan mesej tular mendakwa beliau akan melompat parti menyokong UMNO membentuk kerajaan negeri di Perak. Beliau yang juga Ketua Penerangan Parti Keadilan Rakyat (PKR) Perak dalam satu mesej ringkas kepada Sinar Harian menjelaskan perkara itu tidak benar sama sekali.']

In [8]:
%%time

model.segment([string_hard, string_socialmedia], beam_search = False)

CPU times: user 2.87 s, sys: 444 ms, total: 3.32 s
Wall time: 819 ms


['IPOH - Ahli Dewan Undangan Negeri (ADUN) Hulu Kinta, Muhamad Arafat Varisai Mahamad menafikan mesej tular mendakwa beliau akan melompat parti menyokong UMNO membentuk kerajaan negeri di Perak. Beliau yang juga Ketua Penerangan Parti Keadilan Rakyat (PKR) Perak dalam satu mesej ringkas kepada Sinar Harian menjelaskan perkara itu tidak benar sama sekali.',
 'aq xsukalah ape yg tejadid dekat mamat tu']

In [9]:
%%time

quantized_model.segment([string1, string2, string3, string4], beam_search = False)

CPU times: user 1.09 s, sys: 453 ms, total: 1.55 s
Wall time: 817 ms


['husein suka makan ayam, dia sgt risaukan',
 'dr mahathir sangat menekankan budaya budak zaman sekarang',
 'cerita tun najib razak',
 'Tun M sukakan']

In [10]:
%%time

quantized_model.segment([string_hard, string_socialmedia], beam_search = False)

CPU times: user 3.12 s, sys: 558 ms, total: 3.68 s
Wall time: 891 ms


['IPOH - Ahli Dewan Undangan Negeri (ADUN) Hulu Kinta, Muhamad Arafat Varisai Mahamad menafikan mesej tular mendakwa beliau akan melompat parti menyokong UMNO membentuk kerajaan negeri di Perak. Beliau yang juga Ketua Penerangan Parti Keadilan Rakyat (PKR) Perak dalam satu mesej ringkas kepada Sinar Harian menjelaskan perkara itu tidak benar sama sekali.',
 'aq xsukalah ape yg tejadid dekat mamat tu']